# Sistema Integrado de Trading WDO

Este notebook demonstra o funcionamento completo do sistema, integrando:
1. Carregamento e preparação de dados
2. Análise técnica e fundamental
3. Modelos de ML e ensembles
4. Agentes de IA para otimização
5. Simulação e avaliação de resultados

In [ ]:
import osimport sysfrom pathlib import Pathimport pandas as pdimport numpy as npfrom datetime import datetime, timedeltaimport matplotlib.pyplot as pltimport seaborn as snsfrom dotenv import load_dotenv
# Configura caminhosPROJECT_ROOT = Path(os.getcwd()).parentif str(PROJECT_ROOT) not in sys.path:    sys.path.append(str(PROJECT_ROOT))
# Carrega variáveis de ambienteload_dotenv()
# Configura visualizaçõesplt.style.use('seaborn')pd.set_option('display.max_columns', None)
print('Configuração concluída!')

## 1. Carregamento e Preparação dos Dados

Vamos carregar os dados do WDO e preparar as features.

In [ ]:
# Importa módulos necessáriosfrom src.data.market_data_loader import MarketDataLoaderfrom src.ml.features.feature_engineering import FeatureEngineering
# Carrega dadosloader = MarketDataLoader('src/data/candles.db')data = loader.load_data()
print(f'Dados carregados: {len(data)} registros')print('
Primeiros registros:')print(data.head())

In [ ]:
# Prepara featuresengineer = FeatureEngineering()features = engineer.prepare_features(data)
print('
Features criadas:')for col in features.columns:    print(f'- {col}')

## 2. Configuração dos Agentes

Vamos configurar os agentes que auxiliarão na otimização do sistema.

In [ ]:
from src.agents.market_agent import RiskManagementAgent, MarketRegimeAgent, FeatureSelectionAgentfrom src.agents.agent_coordinator import AgentCoordinator
# Inicializa agentesrisk_agent = RiskManagementAgent(max_drawdown=-0.05)regime_agent = MarketRegimeAgent(window_size=20)feature_agent = FeatureSelectionAgent(    initial_features=features.columns.tolist())
# Configura coordenadorcoordinator = AgentCoordinator([    risk_agent,    regime_agent,    feature_agent])

## 3. Treinamento dos Modelos

Agora vamos treinar os modelos base e o ensemble.

In [ ]:
from src.ml.models.lstm_model import LSTMModelfrom src.ml.ensembles.stacking_ensemble import StackingEnsemblefrom sklearn.model_selection import train_test_split
# Prepara dados para MLX = features.valuesy = np.where(data['close'].pct_change().shift(-1) > 0, 1, 0)[:-1]
# Divide dadosX_train, X_test, y_train, y_test = train_test_split(    X, y, test_size=0.2, shuffle=False)
# Treina LSTMlstm = LSTMModel()lstm_history = lstm.train(X_train, y_train)
print('Treinamento LSTM concluído')

In [ ]:
# Configura e treina ensemblefrom sklearn.ensemble import RandomForestClassifierfrom sklearn.linear_model import LogisticRegression
base_models = [    lstm,    RandomForestClassifier(n_estimators=100),    LogisticRegression()]
ensemble = StackingEnsemble(    base_models=base_models,    meta_model=LogisticRegression())
ensemble.fit(X_train, y_train)print('Treinamento do ensemble concluído')

## 4. Simulando o Sistema Completo

Vamos executar uma simulação do sistema completo.

In [ ]:
from src.agents.simulation import AgentSimulator, SimulationConfig
# Configura simulaçãoconfig = SimulationConfig(    duration=100,    initial_capital=100000,    risk_levels=[-0.05, -0.03, -0.02],    volatility_regimes=['low', 'medium', 'high'],    feature_sets=[features.columns.tolist()])
simulator = AgentSimulator(config)result = simulator.run_simulation()
print('Simulação concluída')

In [ ]:
# Visualiza resultadossimulator.plot_results(result)
print('
Métricas Finais:')for metric, value in result.metrics.items():    print(f'{metric}: {value:.4f}')

## 5. Avaliação de Backtest

Por fim, vamos avaliar o backtest do sistema.

In [ ]:
from src.backtest.monte_carlo import MonteCarloSimulator
# Executa simulações Monte Carlosimulator_mc = MonteCarloSimulator(n_simulations=1000)
# Simula retornosreturns = data['close'].pct_change().dropna()simulation_result = simulator_mc.simulate_returns(    historical_returns=returns,    initial_capital=100000)
print('
Resultados Monte Carlo:')print(f'VaR (95%): {simulation_result.var:.2%}')print(f'CVaR (95%): {simulation_result.cvar:.2%}')
# Plota resultadossimulator_mc.plot_simulations(simulation_result)